In [1]:
#sqlDF = spark.sql("DROP TABLE transactions_date_fix")
#sqlDF = spark.sql("SELECT * , (transaction_date-20170300) as t_dat, (membership_expire_date-20170300) as e_dat FROM transactions_v2_csv ")

#display(sqlDF)
#sqlDF.write.format("parquet").saveAsTable("transactions_date_fix_v2")


In [2]:
sqlDF = spark.sql("SELECT *  FROM transactions_date_fix")


In [3]:
sqlDF.count()

In [4]:
sqlDF = spark.sql("SELECT msno, MAX(t_dat) as transaction_date, first(payment_method_id) as payment_method_id, AVG(payment_plan_days) as payment_plan_days,  first(plan_list_price) as plan_list_price, first(actual_amount_paid) as actual_amount_paid, first(is_auto_renew) as is_auto_renew, MAX(e_dat) as membership_expire_date, min(is_cancel) as is_cancel FROM transactions_date_fix GROUP BY msno ")
display(sqlDF)



In [5]:
sqlDF.count()

In [6]:
sqlDF2 = spark.sql("SELECT msno, AVG(num_25) AS num_25_avg, AVG(num_50) AS num_50_avg, AVG(num_75) AS num_75_avg, AVG(num_985) AS num_985_avg, AVG(num_unq) AS num_unq_avg, AVG(total_secs) AS total_secs_avg FROM user_logs GROUP BY msno ")
display(sqlDF2)

In [7]:
sqlDF2.count()

In [8]:
sqlDF3 = spark.sql("SELECT * FROM train_v2_csv")
final = sqlDF.join(sqlDF2, 'msno', "outer")
final2 = final.join(sqlDF3, 'msno', "outer")
final3 = final2.na.drop(subset=["total_secs_avg"])
final4 = final3.na.drop(subset=["is_churn"])
final5 = final4.na.drop(subset=["transaction_date"])


In [9]:
display(final4)

In [10]:
sqlDF3.count()

In [11]:
final3.count()

In [12]:
final4.count()

In [14]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["num_25_avg", "num_50_avg", "num_75_avg", "num_unq_avg", "transaction_date", "payment_method_id", "payment_plan_days", "plan_list_price",  "actual_amount_paid", "is_auto_renew", "membership_expire_date", "total_secs_avg"], outputCol="features")
final_df = assembler.transform(final5)
final_df.select("features").show(4, False)

In [15]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression


# Train a GBT model.
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial", labelCol="is_churn")

lrModel = mlr.fit(final_df)


# Make predictions.
#predictions = model.transform(testData)

# Select example rows to display.
#predictions.select("prediction", "indexedLabel", "features").show(200)

# Select (prediction, true label) and compute test error
#evaluator = MulticlassClassificationEvaluator(
#    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
#accuracy = evaluator.evaluate(predictions)
#print("Test Error = %g" % (1.0 - accuracy))

#gbtModel = model.stages[2]
#print(gbtModel)  # summary only

In [16]:
#print("Test Error = %g" % (1.0 - accuracy))

In [17]:
print("Multinomial coefficients: " + str(lrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(lrModel.interceptVector))

In [18]:
sqlDFa = spark.sql("SELECT msno, MAX(t_dat) as transaction_date, first(payment_method_id) as payment_method_id, AVG(payment_plan_days) as payment_plan_days,  first(plan_list_price) as plan_list_price, first(actual_amount_paid) as actual_amount_paid, first(is_auto_renew) as is_auto_renew, MAX(e_dat) as membership_expire_date, min(is_cancel) as is_cancel FROM transactions_date_fix_v2 GROUP BY msno ")
display(sqlDFa)

sqlDF2a = spark.sql("SELECT msno, AVG(num_25) AS num_25_avg, AVG(num_50) AS num_50_avg, AVG(num_75) AS num_75_avg, AVG(num_985) AS num_985_avg, AVG(num_unq) AS num_unq_avg, AVG(total_secs) AS total_secs_avg FROM user_logs_v2_csv GROUP BY msno ")
display(sqlDF2a)

sqlDF3a = spark.sql("SELECT * FROM sample_submission_v2_csv")
finala = sqlDFa.join(sqlDF2a, 'msno', "outer")
final2a = finala.join(sqlDF3a, 'msno', "outer")
final3a = final2a.na.drop(subset=["total_secs_avg"])
final4a = final3a.na.drop(subset=["is_churn"])
final5a = final4a.na.drop(subset=["transaction_date"])


In [19]:
display(sqlDFa)

In [20]:
from pyspark.ml.feature import VectorAssembler

assemblera = VectorAssembler(
    inputCols=["num_25_avg", "num_50_avg", "num_75_avg", "num_unq_avg", "transaction_date", "payment_method_id", "payment_plan_days", "plan_list_price",  "actual_amount_paid", "is_auto_renew", "membership_expire_date", "total_secs_avg"], outputCol="features")
final_dfa = assemblera.transform(final5a)
final_dfa.select("features").show(4, False)

In [21]:
#featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=10).fit(final_df)
predictionsa = lrModel.transform(final_dfa)

In [22]:
display(predictionsa)

In [23]:
predictionsa.count()

In [24]:
predictionsa.registerTempTable("predictions")
msnochurn = sql("SELECT msno, prediction FROM predictions")

In [25]:
sqlDF11 = spark.sql("SELECT msno, is_churn FROM sample_submission_v2_csv")


display(sqlDF11)


In [26]:
display(msnochurn)

In [27]:
EOLL = msnochurn.join(sqlDF11, 'msno', "outer")
display(EOLL)

In [28]:
#spark.sql("DROP TABLE output")
EOLL.registerTempTable("done_tabb")

tofile = sql("SELECT msno, COALESCE(is_churn, 0) + COALESCE(prediction, 0) as is_churn FROM done_tabb")


In [30]:
display(tofile)

In [31]:


done.write.format("parquet").saveAsTable("outbounddata")

In [32]:
tocsv = spark.sql("SELECT msno, COALESCE(is_churn, 0) + COALESCE(prediction, 0) as is_churn FROM output")
display(tocsv)

In [33]:
tocsv.count()

In [34]:
tocsv.write.csv('mycsv.csv')